LM with retrieval

Idea: verify that retrieval can improve prediction quality for small models

In [1]:
import jax
from typing import Any, Callable, Sequence, Optional
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze
from flax import linen as nn

In [2]:
from transformers import BertTokenizer, FlaxBertForPreTraining, FlaxBertModel

Retrieval model, dual encoder to retireve documents.

In [3]:
# A pretrained Hugging face model that is used for retrieval
MODEL_TYPE = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)
bert_encoder = FlaxBertModel.from_pretrained(MODEL_TYPE)

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Interpreter Host
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: {('pooler', 'dense', 'bias'), ('pooler', 'dense', 'kernel')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Try inference of the model
inputs = tokenizer("Hello, my dog is cute", return_tensors="jax")
outputs = bert_encoder(**inputs)
outputs.pooler_output.shape

(1, 768)

In [5]:
class DocumentEncoder(nn.Module):
    dimensions: Sequence[int]
    encoder: FlaxBertModel

    @nn.compact
    def __call__(self, input):
        x = self.encoder(**inputs).pooler_output
        for i, feat in enumerate(self.dimensions):
            x = nn.Dense(feat, name=f'layers_{i}')(x)
            if i != len(self.dimensions) - 1:
                x = nn.relu(x)
        return x

model = DocumentEncoder(dimensions=[768,768], encoder=bert_encoder)
model.setup

<bound method Module.setup of DocumentEncoder(
    # attributes
    dimensions = [768, 768]
    encoder = <transformers.models.bert.modeling_flax_bert.FlaxBertModel object at 0x7fb751b2dd30>
)>

In [6]:
# initialize the model parameters
params = model.init(random.PRNGKey(0), inputs)


In [7]:
model.apply(params, inputs)

DeviceArray([[-0.339502  ,  0.0432492 ,  0.01497814, -0.11061206,
              -0.02616203, -0.02092332,  0.1959934 , -0.08281386,
              -0.10037724,  0.06958833,  0.10271972,  0.33912143,
              -0.16751938, -0.20066957, -0.12156841, -0.2113133 ,
              -0.20840159, -0.04508257, -0.01584379,  0.21792877,
               0.09600222,  0.39484373,  0.45374483,  0.34563887,
               0.08924327, -0.12511891,  0.46351707,  0.05140289,
              -0.33967775, -0.08923145,  0.00472189,  0.22341143,
              -0.17614679,  0.07717368,  0.05756273,  0.20330286,
              -0.03640734,  0.09931888,  0.01950785,  0.1644101 ,
              -0.22386608,  0.02179904,  0.22025782, -0.02451881,
              -0.23692493, -0.39110643, -0.16336784, -0.0746641 ,
               0.54974484,  0.12904316,  0.08858219, -0.3262225 ,
              -0.08783709, -0.09947664, -0.05356479,  0.2096961 ,
              -0.3642194 , -0.13723595,  0.26316294,  0.15356146,
          

Preparing Dataset, using wikipedia dataset for training. The retrieval model predicts extracted text from a wikipedia paragraph.

In [8]:
from datasets import load_dataset

dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')

In [9]:
import random as python_random

def extract_random_sentence(example):
    """Sample one sentence from a paragraph."""
    sentences = example['text'].split('.')
    sentences_len = len(sentences)
    if (sentences_len < 2):
        return {'paragraph': '', 'sample': ''}
    sampled_id = python_random.randint(0, sentences_len-1)
    paragraph = ' '.join(sentences[:sampled_id] + sentences[sampled_id+1:])
    sampled_sentence = sentences[sampled_id]
    if not sampled_sentence.strip() or not paragraph.strip():
        return {'paragraph': '', 'sample': ''}

    return {'paragraph': paragraph, 'sample': sampled_sentence}

def filter_callback(example):
    """Filters out examples with an empty paragraph."""
    return len(example['paragraph']) > 0

In [41]:
def DatastreamTokenize():
    """Creates a callback that performs tokenization and also batching"""
    tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)
    def TokenizerCallback(example):
        build_tokenized_map = lambda prefix: {prefix+'_'+key: value for key, value in tokenizer(example['paragraph']).items()}
        return build_tokenized_map('paragraph').update(build_tokenized_map('sample'))
    return TokenizerCallback

In [39]:
example = extract_random_sentence({'text':'Hello World. Good bye'})

DatastreamTokenize()(example)

In [28]:
prepared_dataset = dataset.map(extract_random_sentence).filter(filter_callback).map(DatastreamTokenize(), batched=True, batch_size=32)


  0%|          | 0/385 [00:00<?, ?ba/s]


TypeError: Provided `function` which is applied to all elements of table returns a `dict` of types {[type(x) for x in processed_inputs.values()]}. When using `batched=True`, make sure provided `function` returns a `dict` of types like `{allowed_batch_return_types}`.

In [27]:
prepared_dataset.shape, dataset.shape

((12313, 3), (36718, 1))